In [3]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import json
import pandas as pd

In [3]:
my_key = 'apikey'

In [1]:
def get_api(keyword):

    api_key = my_key


    url = 'https://api.nal.usda.gov/fdc/v1/foods/search?api_key=' + api_key +'&query=' +keyword

    response = requests.get(url)

    data = json.loads(response.content)

    return data

### 영양소 기본 단위 확인

In [112]:
url = 'https://api.nal.usda.gov/fdc/v1/foods?fdcIds=2012128&fdcIds=170999&fdcIds=168849&nutrients=203&nutrients=204&nutrients=205&api_key='+ my_key

response = requests.get(url)

data = json.loads(response.content)

data

[{'discontinuedDate': '',
  'foodComponents': [],
  'foodAttributes': [{'id': 2203573,
    'value': 9,
    'name': 'Added Package Weight'}],
  'foodPortions': [],
  'fdcId': 2012128,
  'description': 'BANANA',
  'publicationDate': '10/28/2021',
  'foodNutrients': [{'type': 'FoodNutrient',
    'nutrient': {'id': 1003,
     'number': '203',
     'name': 'Protein',
     'rank': 600,
     'unitName': 'g'},
    'foodNutrientDerivation': {'id': 70,
     'code': 'LCCS',
     'description': 'Calculated from value per serving size measure'},
    'id': 26035654,
    'amount': 12.5},
   {'type': 'FoodNutrient',
    'nutrient': {'id': 1005,
     'number': '205',
     'name': 'Carbohydrate, by difference',
     'rank': 1110,
     'unitName': 'g'},
    'foodNutrientDerivation': {'id': 70,
     'code': 'LCCS',
     'description': 'Calculated from value per serving size measure'},
    'id': 26035656,
    'amount': 40.62},
   {'type': 'FoodNutrient',
    'nutrient': {'id': 1004,
     'number': '204',
 

In [109]:
for nu in data['foods']:
    for n in nu['foodNutrients']:
        nutrient_id = n['nutrientId']
        nutrient_name = n['nutrientName']
        nutrient_num = n['nutrientNumber']
        nutrient_unit = n['unitName']

        if nutrient_id not in nutrients_info['n_id']:

            row = [nutrient_id, nutrient_name, nutrient_num, nutrient_unit]
            series = pd.Series(row, index = nutrients_info.columns)
            nutrients_info = nutrients_info.append(series, ignore_index = True)


C:\Users\USER\AppData\Local\Temp\ipykernel_17472\1234570474.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nutrients_info = nutrients_info.append(series, ignore_index = True)
C:\Users\USER\AppData\Local\Temp\ipykernel_17472\1234570474.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nutrients_info = nutrients_info.append(series, ignore_index = True)
C:\Users\USER\AppData\Local\Temp\ipykernel_17472\1234570474.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nutrients_info = nutrients_info.append(series, ignore_index = True)
C:\Users\USER\AppData\Local\Temp\ipykernel_17472\1234570474.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat

In [110]:
nutrients_info = nutrients_info.drop_duplicates(['n_id']).reset_index(drop=True)
nutrients_info

,n_id,n_name,n_number,unit
0,1003,Protein,203,G
1,1004,Total lipid (fat),204,G
2,1005,"Carbohydrate, by difference",205,G
3,1008,Energy,208,KCAL
4,2000,"Sugars, total including NLEA",269,G
...,...,...,...,...
185,2010,MUFA 17:1 c,825,G
186,2026,PUFA 20:2 c,840,G
187,1410,PUFA 21:5,857,G
188,1310,"TFA 18:2 t,t",669,G


In [98]:
nutrients_info.to_csv('./nutrients_info.csv', index=False)

In [112]:
nutrients_info

,n_id,n_name,n_number,unit
0,1003,Protein,203,G
1,1004,Total lipid (fat),204,G
2,1005,"Carbohydrate, by difference",205,G
3,1008,Energy,208,KCAL
4,2000,"Sugars, total including NLEA",269,G
...,...,...,...,...
185,2010,MUFA 17:1 c,825,G
186,2026,PUFA 20:2 c,840,G
187,1410,PUFA 21:5,857,G
188,1310,"TFA 18:2 t,t",669,G


### 상품명을 검색해 API에서 영양소 정보 가져오기

In [5]:
merged_df = pd.read_csv('./데이터/real_final_merged_data.csv')
merged_df.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle,department
0,1,49302,1,1,Bulgarian Yogurt,120,16,112108,train,4,4,10,9.0,yogurt,dairy eggs
1,816049,49302,7,1,Bulgarian Yogurt,120,16,47901,train,14,4,6,16.0,yogurt,dairy eggs
2,1242203,49302,1,1,Bulgarian Yogurt,120,16,2993,train,15,0,7,7.0,yogurt,dairy eggs
3,1383349,49302,11,1,Bulgarian Yogurt,120,16,41425,train,4,3,8,14.0,yogurt,dairy eggs
4,1787378,49302,8,0,Bulgarian Yogurt,120,16,187205,train,5,4,14,30.0,yogurt,dairy eggs


In [6]:
menu = list(merged_df['product_name'].unique())
p_num = list(merged_df['product_id'].unique())


print(len(menu), len(p_num))

25386 25386


In [163]:
columns = ['Product_id', 'FoodName', 'Energy', 'Carbohydrate', 'Sugar', 'Protein', 'Fat', 'Water', 'Vitamin A', 'Vitamin B-6', 'Vitamin B-12', 'Vitamin C', 'Vitamin D', 'Ca', 'Na', 'P', 'K', 'Cl', 'Mg']
food_nutrient_df = pd.DataFrame(columns=columns)

for idx in range(len(menu)):
    data = get_api(menu[idx])
    
    #영양소 값 초기화 (영양 정보가 없는 상품 대비하기 위해서)
    energy = 0.0
    carbohd = 0.0
    protein = 0.0
    fat = 0.0
    sugar = 0.0
    water = 0.0
    vit_a = 0.0
    vit_b6 = 0.0
    vit_b12 = 0.0
    vit_c = 0.0
    vit_d = 0.0
    na = 0.0
    ca = 0.0
    p = 0.0
    mg = 0.0
    k = 0.0
    s = 0.0
    cl = 0.0

    #영양소가 없는 상품에서 발생하는 오류 대비 try문으로 실행
    try:
        for n in data['foods'][0]['foodNutrients']:
            if n['nutrientNumber'] == '203':
                protein = n['value']
            elif n['nutrientNumber'] == '204':
                fat = n['value']
            elif n['nutrientNumber'] == '205':
                carbohd = n['value']
            elif n['nutrientNumber'] == '208' or n['nutrientNumber'] == '268': # 268 KJ
                if n['nutrientNumber'] == '208':
                    energy = n['value']
                elif n['nutrientNumber'] == '268':
                    energy = n['value'] * 0.239
            elif n['nutrientNumber'] == '269':
                sugar = n['value']
            elif n['nutrientNumber'] == '301':
                ca = n['value']
            elif n['nutrientNumber'] == '307':
                na = n['value']
            elif n['nutrientNumber'] == '318':
                vit_a = n['value']
            elif n['nutrientNumber'] == '324':
                vit_d = n['value']
            elif n['nutrientNumber'] == '401':
                vit_c = n['value']
            elif n['nutrientNumber'] == '415':
                vit_b6 = n['value']
            elif n['nutrientNumber'] == '418':
                vit_b12 = n['value']
            elif n['nutrientNumber'] == '255':
                water = n['value']
            elif n['nutrientNumber'] == '305':
                p = n['value']
            elif n['nutrientNumber'] == '306':
                k = n['value']
            elif n['nutrientNumber'] == '421':
                cl = n['value']
            elif n['nutrientNumber'] == '304':
                mg = n['value']
    except:
        pass

    row = [p_num[idx], menu[idx], energy, carbohd, sugar, protein, fat, water, vit_a, vit_b6, vit_b12, vit_c, vit_d, ca, na, p, k, cl, mg]
    series = pd.Series(row, index = food_nutrient_df.columns)
    food_nutrient_df = food_nutrient_df.append(series, ignore_index = True)

C:\Users\USER\AppData\Local\Temp\ipykernel_17472\993709494.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  food_nutrient_df = food_nutrient_df.append(series, ignore_index = True)
C:\Users\USER\AppData\Local\Temp\ipykernel_17472\993709494.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  food_nutrient_df = food_nutrient_df.append(series, ignore_index = True)
C:\Users\USER\AppData\Local\Temp\ipykernel_17472\993709494.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  food_nutrient_df = food_nutrient_df.append(series, ignore_index = True)
C:\Users\USER\AppData\Local\Temp\ipykernel_17472\993709494.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use panda

KeyboardInterrupt: 

In [164]:
food_nutrient_df

,Product_id,FoodName,Energy,Carbonhydrate,Sugar,Protein,Fat,Water,Vitamin A,Vitamin B-6,Vitamin B-12,Vitamin C,Vitamin D,Ca,Na,P,K,Cl,Mg
0,49302,Bulgarian Yogurt,38.0,3.39,2.54,3.39,0.21,0.0,85.0,0.0,0.0,0.0,42.0,106,44.0,0.0,0.0,0.0,0.0
1,6104,Whole Milk Plain Yogurt,61.0,4.66,4.66,3.47,3.25,87.9,99.0,0.032,0.37,0.5,2.0,121,46.0,95.0,155,15.2,12.0
2,45521,Organic Plain Cream Top Yogurt,66.0,9.69,5.29,2.64,1.76,84.7,0.0,0.0,0.0,13.2,53.0,132,13.0,0.0,0.0,0.0,0.0
3,39475,Total Greek Strained Yogurt,57.0,3.96,3.96,10.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,110,37.0,0.0,0.0,0.0,0.0
4,44008,Organic Greek Whole Milk Blended Vanilla Bean ...,106,12.3,12.0,7.33,3.0,76.8,162,0.049,0.5,0.0,0.0,100,37.0,109,113,14.3,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6615,2743,Organic Almonds Sprouted In Orange Juice,45.0,10.2,8.26,0.6,0.3,88.5,55.0,0.054,0.0,62.5,0.0,12.0,0.0,11.0,184,6.2,9.0
6616,29012,Go Walnuts! Organic Walnuts,884,0.0,0.0,0.0,100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0
6617,44969,Brazil Nuts,659,11.7,2.33,14.3,67.1,3.42,0.0,0.101,0.0,0.7,0.0,160,3.0,725,659,28.8,376
6618,16766,Organic Freeze Dried Bananas and Strawberries,311,64.3,0.0,21.2,3.5,2.0,68300.0,2.0,0.0,660,0.0,813,70.0,518,2960.0,0.0,640


In [8]:
outlier_removed_merged_df = pd.read_csv('./데이터/data_outlier_remove_high5_mid50.csv')

In [9]:
product_list = list(outlier_removed_merged_df['product_id'].unique())


In [6]:
#파일 저장
p_nutrients = pd.read_csv('./데이터/food_nutrient.csv')
p_nutrients.head(5)

,Product_id,FoodName,Energy,Carbohydrate,Sugar,Protein,Fat,Water,Vitamin A,Vitamin B-6,Vitamin B-12,Vitamin C,Vitamin D,Ca,Na,P,K,Cl,Mg
0,49302,Bulgarian Yogurt,38.000,3.39,2.54,3.39,0.21,0.0,85.0,0.000,0.00,0.0,42.0,106.0,44.0,0.0,0.0,0.0,0.0
1,6104,Whole Milk Plain Yogurt,61.423,4.66,4.66,3.47,3.25,87.9,99.0,0.032,0.37,0.5,2.0,121.0,46.0,95.0,155.0,15.2,12.0
2,45521,Organic Plain Cream Top Yogurt,65.964,9.69,5.29,2.64,1.76,84.7,0.0,0.000,0.00,13.2,53.0,132.0,13.0,0.0,0.0,0.0,0.0
3,39475,Total Greek Strained Yogurt,57.000,3.96,3.96,10.10,0.00,0.0,0.0,0.000,0.00,0.0,0.0,110.0,37.0,0.0,0.0,0.0,0.0
4,44008,Organic Greek Whole Milk Blended Vanilla Bean ...,105.399,12.30,12.00,7.33,3.00,76.8,162.0,0.049,0.50,0.0,0.0,100.0,37.0,109.0,113.0,14.3,10.0


### outlier를 제거한 DataFrame의 product_list에 없는 상품 영양소 정보 제거

In [43]:
p_nutrients_2 = p_nutrients.copy()

In [41]:
idx = []
for i in range(len(p_nutrients['Product_id'])):
    if p_nutrients['Product_id'][i] not in product_list:
        idx.append(i)

In [44]:
for i in idx:
    p_nutrients_2.drop(i, axis = 0, inplace = True)

In [58]:
p_nutrients_2 = p_nutrients_2.reset_index(drop = True)

In [60]:
p_nutrients_2.to_csv('./데이터/new_food_nutrient.csv', index = False)

### mysql에 DB 생성

In [48]:
import pymysql

In [49]:
conn_db = pymysql.connect(
    user = 'root',
    passwd = '1234',
    host = '127.0.0.1',
    db = 'snupfy',
    charset = 'utf8'
)

In [50]:
cursor = conn_db.cursor(pymysql.cursors.DictCursor)

In [66]:
for idx in range(len(p_nutrients_2)):
    pid = p_nutrients_2['Product_id'][idx]
    energy = round(p_nutrients_2['Energy'][idx],1)
    carbohd = round(p_nutrients_2['Carbohydrate'][idx],1)
    sugar = round(p_nutrients_2['Sugar'][idx],1)
    protein = round(p_nutrients_2['Protein'][idx],1)
    fat = round(p_nutrients_2['Fat'][idx],1)
    water = p_nutrients_2['Water'][idx]
    vita = p_nutrients_2['Vitamin A'][idx]
    vitb6 = round(p_nutrients_2['Vitamin B-6'][idx],1)
    vitb12 = round(p_nutrients_2['Vitamin B-12'][idx],1)
    vitc = p_nutrients_2['Vitamin C'][idx]
    vitd = p_nutrients_2['Vitamin D'][idx]
    ca = p_nutrients_2['Ca'][idx]
    na = p_nutrients_2['Na'][idx]
    p = p_nutrients_2['P'][idx]
    k = p_nutrients_2['K'][idx]
    cl = p_nutrients_2['Cl'][idx]
    mg = p_nutrients_2['Mg'][idx]

    qry = '''
    insert into nutrient values ({},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{});
    '''.format(pid, energy, carbohd, sugar, protein, fat, water, vita, vitb6, vitb12, vitc, vitd, ca, na, p, k, cl, mg)
    cursor.execute(qry)

conn_db.commit()